# Week 3 Analysis

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
import scipy.stats as stats
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

## Tuning Functions

In [ ]:
def gridSearch(X, Y, repeat, n_splits, scorer, mod, hyperparameters,  n_jobs=None, stratify=True):
    # GridSearch Wrapper Fucntion
    print("Number of repeats run is: " + str(repeat))
    dfL = []
    for i in range(0,repeat):
        if stratify==True:
            cv = StratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
        else:
            cv = KFold(n_splits=n_splits, random_state=i, shuffle=True)
        boosted_grid = GridSearchCV(mod, hyperparameters, scoring=scorer, cv=cv, verbose=0, refit=True, error_score=np.nan, return_train_score=True, n_jobs=n_jobs) #n_jobs=n_jobs,
        grid_fit = boosted_grid.fit(X, Y)
        DF = pd.DataFrame(grid_fit.cv_results_)
        DF['Iteration'] = i
        dfL.append(DF)
    DFall = pd.concat(dfL)
    return DFall


def OverSampler(parentDIR,df, xfilename, yfilename):
    # Oversample small class to balanced data
    os.chdir(parentDIR)
    df = df.dropna()
    X = df.iloc[:,4:].copy()
    X['TimeGroup'] = df.TimeGroup.copy()
    X = pd.get_dummies(X)
    Y = df.Status.copy()

    ros = RandomOverSampler(random_state=0)
    Xoversampled, Yoversampled = ros.fit_resample(X, Y)

    if not os.path.isdir('OverSampled'):
        os.makedirs('OverSampled')
    os.chdir('OverSampled')

    Xoversampled.to_csv(xfilename, sep='\t')
    Yoversampled.to_csv(yfilename, sep='\t')
    return Xoversampled, Yoversampled

## Data Loading

In [ ]:
parentdir = os.getcwd()

X = pd.read_csv('W3_predictor.txt', sep='\t', index_col=0)
Y = pd.read_csv('W3_response.txt', sep='\t', index_col=0).squeeze()

## Model Tuning

In [ ]:
os.chdir(parentdir)
if not os.path.isdir('Results'):
    os.makedirs('Results')
os.chdir('Results')
RAND=np.random.RandomState(4)

param_space = {'C':np.logspace(-5,2,4),
                'l1_ratio':[float(x) for x in np.linspace(0.1,0.9,4)]}
enet = LogisticRegression(penalty = 'elasticnet', solver = 'saga', max_iter=int(1e6))

Enet = gridSearch(X=X, Y=Y, repeat=200, n_splits=10, scorer='roc_auc', mod=enet, hyperparameters=param_space, n_jobs=6, stratify=True) #, n_jobs=numCores
Enet.to_csv('W3_EnetBurkPx_GridSearch.txt', sep='\t')
Enet['params2'] = Enet['params'].astype(str)
print('Finished Enet')


param_space = {'C':np.logspace(-5,3,10),
                'penalty':['l1', 'l2']}
lr = LogisticRegression( solver = 'saga', max_iter=int(1e6))

LR = gridSearch(X=X, Y=Y, repeat=200, n_splits=10, scorer='roc_auc', mod=lr, hyperparameters=param_space, n_jobs=6, stratify=True) #, n_jobs=numCores
LR.to_csv('W3_LRBurkPx_GridSearch.txt', sep='\t')
LR['params2'] = LR['params'].astype(str)
print('Finished LR')


param_space = {'learning_rate': np.logspace(-3, -1,4),
                'n_estimators': [25, 50],
                'subsample': np.linspace(0.2,0.9,3),
                'colsample_bytree':np.linspace(0.05, 0.5, 3),
                'max_depth':[int(x) for x in np.linspace(3,20,3)]}

xgb =  XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=RAND) #, scale_pos_weight=len(Y==0)/len(Y==1) BinaryFocalLoss

LR = gridSearch(X=X, Y=Y, repeat=200, n_splits=10, scorer='roc_auc', mod=xgb, hyperparameters=param_space, n_jobs=6, stratify=True) #, n_jobs=numCores
LR.to_csv('W3_XGBBurkPx_GridSearch.txt', sep='\t')
LR['params2'] = LR['params'].astype(str)

## Preliminary Figure Analysis
### Enet

In [ ]:
LR = pd.read_csv('W3_EnetBurkPx_GridSearch.txt', sep='\t', index_col=0)
LR['params2'] = LR['params'].astype(str)
fig, ax = plt.subplots()
sns.boxplot(data=LR, y= 'mean_test_score', x='params2', ax=ax)
plt.xticks(rotation=90)

### Penalized Logit

In [ ]:
LR = pd.read_csv('W3_LRBurkPx_GridSearch.txt', sep='\t', index_col=0)
LR['params2'] = LR['params'].astype(str)
fig, ax = plt.subplots()
sns.boxplot(data=LR, y= 'mean_test_score', x='params2', ax=ax)
plt.xticks(rotation=90)

### XGB

In [ ]:
LR = pd.read_csv('W3_XGBBurkPx_GridSearch.txt', sep='\t', index_col=0)
LR['params2'] = LR['params'].astype(str)
fig, ax = plt.subplots()
sns.boxplot(data=LR, y= 'mean_test_score', x='params2', ax=ax)
plt.xticks(rotation=90)

## Overall Model Comparison

In [ ]:
os.chdir(parentdir)
os.chdir('Results')

LR = pd.read_csv('W3_EnetBurkPx_GridSearch.txt', sep='\t', index_col=0)
W1Enet = LR.loc[LR['params'] =="{'C': 0.0021544346900318843, 'l1_ratio': 0.6333333333333333}"]
W1Enet['Model']  = 'ElasticNet'

LR = pd.read_csv('W3_LRBurkPx_GridSearch.txt', sep='\t', index_col=0)
W1LASSOWeak=LR.loc[LR['params']== "{'C': 1000.0, 'penalty': 'l1'}"].copy()
W1LASSOWeak['Model'] = 'LASSO_Weak'

W1LASSOstrong=LR.loc[LR['params']== "{'C': 0.004641588833612777, 'penalty': 'l1'}"].copy()
W1LASSOstrong['Model'] = 'LASSO_Strong'

W1RIDGEWeak=LR.loc[LR['params']== "{'C': 1000.0, 'penalty': 'l2'}"].copy()
W1RIDGEWeak['Model'] = 'Ridge_Weak'

W1RIDGEstrong=LR.loc[LR['params']== "{'C': 0.004641588833612777, 'penalty': 'l2'}"].copy()
W1RIDGEstrong['Model'] = 'Ridge_Strong'


LR = pd.read_csv('W3_XGBBurkPx_GridSearch.txt', sep='\t', index_col=0)
W1XGB = LR.loc[LR['params']=="{'colsample_bytree': 0.05, 'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.55}"]
W1XGB['Model'] = 'XGBoost'
BEST = pd.concat([W1LASSOWeak, W1LASSOstrong, W1RIDGEWeak, W1RIDGEstrong, W1Enet, W1XGB])

BEST['params2'] = BEST['params'].astype(str)

sns.set_style('darkgrid')
sns.set_palette('colorblind')
fig, ax = plt.subplots()
sns.boxplot(data=BEST, y= 'mean_test_score', x='Model', hue=None, ax=ax)
plt.xticks(rotation=45)
ax.set_title('Week3 Model Comparions')

fig.savefig('Week3_Model_Comparison_Oversampled_rocAUC.png', bbox_inches='tight', dpi=300)

## Analyzing Validation Set
### Enet

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
os.chdir('../../ValidationSet')

df = pd.read_csv('ValidationSet.txt', sep='\t', index_col=0)
Week3 = df.loc[(df['TimeGroup']=='Week3')|(df['TimeGroup']=='Healthy')]

yVAL = Week3.Status.copy()
yVAL.replace('Melioid', 1, inplace=True)
yVAL.replace('Negative', 0, inplace=True)

XVAL = Week2.iloc[:,4:].copy()
XVAL = pd.get_dummies(XVAL)

ros = RandomUnderSampler(random_state=0)
Xunder, Yunder = ros.fit_resample(XVAL, yVAL)


enet = LogisticRegression(penalty = 'elasticnet', C=0.0021544346900318843, l1_ratio=0.6333333333333333, solver = 'saga', max_iter=int(1e6)).fit(Xunder, Yunder)
# roc_auc_score(y, clf.predict_proba(X)[:, 1])
roc_auc_score(Yunder, enet.predict_proba(Xunder)[:, 1])
# roc_auc_score(yVAL, enet.decision_function(XVAL))